In [1]:
# 1.导包
# 1.1在canda下安装pyecharts
# pip install pyecharts

import time
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from pyecharts.charts import Bar3D
from pyecharts.commons.utils import JsCode
import pyecharts.options as opts
import os
os.chdir(r"E:\ProjectCode\PythonProjectCode\pythonProject\HaskMaProject\data")

In [5]:
# 2.导入数据
sheet_names = ['2015','2016','2017','2018','会员等级',]
# sheet_name: str | int | list[IntStrT] | None = 0
sheet_dict = pd.read_excel('sales.xlsx',sheet_name=sheet_names)

In [7]:
# 2.1查看某一个sheet页的基本信息
sheet_dict['2015'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30774 entries, 0 to 30773
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   会员ID    30774 non-null  int64         
 1   订单号     30774 non-null  int64         
 2   提交日期    30774 non-null  datetime64[ns]
 3   订单金额    30774 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 961.8 KB


In [6]:
# 2.2[数据质量查看]查看某一个sheet页的统计信息
sheet_dict['2015'].describe()

,会员ID,订单号,提交日期,订单金额
count,3.077400e+04,3.077400e+04,30774,30774.000000
mean,2.918779e+10,4.020414e+09,2015-07-01 20:55:49.424839168,960.991161
min,2.670000e+02,3.000305e+09,2015-01-01 00:00:00,0.500000
25%,1.944122e+10,3.885510e+09,2015-04-02 00:00:00,59.000000
50%,3.746545e+10,4.117491e+09,2015-07-02 00:00:00,139.000000
75%,3.923593e+10,4.234882e+09,2015-10-01 00:00:00,899.000000
max,3.954613e+10,4.282025e+09,2015-12-31 00:00:00,111750.000000
std,1.385333e+10,2.630510e+08,NaN,2068.107231


In [8]:
# 2.2[数据质量查看]查看全部  2015、2016、2017、2018
# 遍历查看每一页页签的数据信息
for i in sheet_names:
    print(i)        # 每一个sheet页的名字
    print(sheet_dict[i].info)
    print(sheet_dict[i].describe())

2015
<bound method DataFrame.info of               会员ID         订单号       提交日期    订单金额
0      15278002468  3000304681 2015-01-01   499.0
1      39236378972  3000305791 2015-01-01  2588.0
2      38722039578  3000641787 2015-01-01   498.0
3      11049640063  3000798913 2015-01-01  1572.0
4      35038752292  3000821546 2015-01-01    10.1
...            ...         ...        ...     ...
30769  39368100847  4281994827 2015-12-31   828.0
30770       409757  4282010457 2015-12-31   199.0
30771  38380526114  4282017675 2015-12-31   208.0
30772     28074988  4282019440 2015-12-31    89.0
30773  39460363230  4282025309 2015-12-31   719.0

[30774 rows x 4 columns]>
               会员ID           订单号                           提交日期  \
count  3.077400e+04  3.077400e+04                          30774   
mean   2.918779e+10  4.020414e+09  2015-07-01 20:55:49.424839168   
min    2.670000e+02  3.000305e+09            2015-01-01 00:00:00   
25%    1.944122e+10  3.885510e+09            2015-04-02 00:00:00

In [9]:
# 3.对源数据进行处理（空值、极小值） --> 异常值
# 空值：可填充 可删除
# 极小值：低于一元的订单均删除
# 固定时间点：以每年的最后一天作为当年的时间点

In [10]:
# 3.1:删除空值 & 过滤出订单金额 > 1  &  固定时间点 ：以每年的最后一天作为当年的时间点
# for i in sheet_names[:len(sheet_names)-1]:
for i in sheet_names[:-1]:
    # 3.1.1:删除空值 dropna()  删除数据中的空值
    sheet_dict[i] = sheet_dict[i].dropna()
    # 3.1.2:过滤金额
    sheet_dict[i] = sheet_dict[i][sheet_dict[i]['订单金额']>1]
    # 3.1.3:固定时间点：以每年的最后一天作为当年的时间点
    sheet_dict[i]['max_year_date'] = sheet_dict[i]['提交日期'].max()

In [11]:
# 3.2:查看处理后的数据
for i in sheet_names:
    print(sheet_dict[i].info())
    print(sheet_dict[i].describe())

<class 'pandas.core.frame.DataFrame'>
Index: 30574 entries, 0 to 30773
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   会员ID           30574 non-null  int64         
 1   订单号            30574 non-null  int64         
 2   提交日期           30574 non-null  datetime64[ns]
 3   订单金额           30574 non-null  float64       
 4   max_year_date  30574 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2)
memory usage: 1.4 MB
None
               会员ID           订单号                           提交日期  \
count  3.057400e+04  3.057400e+04                          30574   
mean   2.921327e+10  4.020442e+09  2015-07-01 21:10:04.042650624   
min    2.670000e+02  3.000305e+09            2015-01-01 00:00:00   
25%    1.961657e+10  3.885746e+09            2015-04-02 00:00:00   
50%    3.754532e+10  4.117491e+09            2015-07-02 00:00:00   
75%    3.923630e+10  4.234853e+09            2015-10-01 00

In [13]:
# 3.3：把上述的四张表(dataframe)  合并为一个df对象
# sheet_dict['2015']
# sheet_dict['2016']
# sheet_dict['2017']
# sheet_dict['2018']
# list(sheet_dict.values())
# ignore_index=True     # 合并后的表格行号从0开始重新排队，避免混乱
df_merge = pd.concat(list(sheet_dict.values())[:-1],ignore_index=True)
df_merge

,会员ID,订单号,提交日期,订单金额,max_year_date
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31
...,...,...,...,...,...
202822,39229485704,4354225182,2018-12-31,249.0,2018-12-31
202823,39229021075,4354225188,2018-12-31,89.0,2018-12-31
202824,39288976750,4354230034,2018-12-31,48.5,2018-12-31
202825,26772630,4354230163,2018-12-31,3196.0,2018-12-31


In [35]:
# 3.4：为了好区分，给df增加一个year列
df_merge['year'] = df_merge['提交日期'].dt.year
df_merge

,会员ID,订单号,提交日期,订单金额,max_year_date,year,data_interval
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31,2015,364
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31,2015,364
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31,2015,364
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31,2015,364
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31,2015,364
...,...,...,...,...,...,...,...
202822,39229485704,4354225182,2018-12-31,249.0,2018-12-31,2018,0
202823,39229021075,4354225188,2018-12-31,89.0,2018-12-31,2018,0
202824,39288976750,4354230034,2018-12-31,48.5,2018-12-31,2018,0
202825,26772630,4354230163,2018-12-31,3196.0,2018-12-31,2018,0


In [43]:
# 3.5：预处理R字段
#   时间间隔 = 截止时间 - 提交时间     算出用户最近间隔
df_merge['data_interval']=df_merge['max_year_date'] - df_merge['提交日期']
# df_merge
# 现状：  显示 364 days
#    想要     364
# df_merge['data_interval'] = df_merge['data_interval'].dt.days
# df_merge
# <class 'pandas.core.indexes.accessors.TimedeltaProperties'>
# TimedeltaProperties
print(type(df_merge['data_interval'].dt))



<class 'pandas.core.indexes.accessors.TimedeltaProperties'>
<class 'pandas.core.series.Series'>


In [22]:
# 按照年份、会员ID进行分组
#   R：取间隔时间最小值
#   F：订单数量
#   M：订单金额

In [25]:
# as_index = False  不要把分组字段设置为索引
rfm = df_merge.groupby(['year','会员ID'],as_index=False).agg(
    {
        'data_interval':'min',
        '订单号':'count',
        '订单金额':'sum'
    }
)

In [26]:
# 修改列名（方便分析）
rfm.columns = ['year','会员ID','r','f','m']
rfm

,year,会员ID,r,f,m
0,2015,267,197,2,105.0
1,2015,282,251,1,29.7
2,2015,283,340,1,5398.0
3,2015,343,300,1,118.0
4,2015,525,37,3,213.0
...,...,...,...,...,...
148586,2018,39538034299,272,1,49.0
148587,2018,39538034662,189,1,3558.0
148588,2018,39538035729,179,1,3699.0
148589,2018,39545237824,275,1,49.0
